In [1]:
import os
import torchvision
from PIL import Image
from torchvision import datasets, models, transforms
import torch

# number of subprocesses to use for data loading
num_workers = 0
classes=133

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

train_transform = transforms.Compose([
         transforms.RandomRotation(30),
         transforms.RandomHorizontalFlip(),
         transforms.RandomResizedCrop(224),            
         transforms.ToTensor()
         ])

transform = transforms.Compose([
         transforms.Resize(256),                    
         transforms.CenterCrop(224),                
         transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder('dogImages/train', transform=train_transform, target_transform=None, loader=Image.open, is_valid_file=None)
valid_data = torchvision.datasets.ImageFolder('dogImages/valid', transform=transform, target_transform=None, loader=Image.open, is_valid_file=None)
test_data = torchvision.datasets.ImageFolder('dogImages/test', transform=transform, target_transform=None, loader=Image.open, is_valid_file=None)

In [2]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
import torch.nn as nn
import torch.nn.functional as F

# instantiate the CNN
model_transfer = models.inception_v3(pretrained=True)

print(model_transfer)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [7]:
# Deactivate gradient for pretrained layers - Prevents them from training
for param in model_transfer.features.parameters():
    param.requires_grad = False
    
# new layers automatically have requires_grad = True
model.AuxLogits.fc = nn.Linear(768, len(train_data.classes))
model.fc = nn.Linear(2048, len(train_data.classes))


# check to see that your last layer produces the expected number of outputs
print('Output Count: {}'.format(last_layer.out_features))

# move tensors to GPU if CUDA is available
if use_cuda:
    model_transfer.cuda()

AttributeError: 'Inception3' object has no attribute 'features'

In [ ]:
from torchsummary import summary
summary(model_transfer, (3, 52, 52))

In [ ]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm import tqdm

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, starting_loss):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = starting_loss 
    
    for epoch in tqdm(range(1, n_epochs+1)):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)----------------------------------------------------------------
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
           if use_cuda:            
               data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
           output = model(data)
            # calculate the batch loss
           loss = criterion(output, target)
            # update average validation loss 
           valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model




In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from glob import glob

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.5, 0.5, 0.5])
    #std = np.array([0.5, 0.5, 0.5])
    #inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 
    
# how many samples per batch to load
batch_size = 20 #lowered batch size due to running out of memory

# prepare data loaders - had to move them because of changing the batch size
loaders_transfer={}
loaders_transfer['train'] = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers, shuffle=True)
loaders_transfer['valid'] = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)
loaders_transfer['test'] = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers, shuffle=True)

# obtain one batch of training images
dataiter = iter(loaders_transfer['train'])
# Get a batch of training data
# inputs contains 4 images because batch_size=4 for the dataloaders
inputs, classes = next(iter(loaders_transfer['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=None)
print('Training Data inputs {}  Classes{}'.format(inputs.shape, classes.shape))
      
# obtain one batch of training images
dataiter2 = iter(loaders_transfer['valid'])
# Get a batch of training data
# inputs contains 4 images because batch_size=4 for the dataloaders
inputs, classes = next(iter(loaders_transfer['valid']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=None)
print('Validation Data inputs {}  Classes{}'.format(inputs.shape, classes.shape))

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_transfer = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9, nesterov=True)

training_from_base=True
update_training=False

In [ ]:
#Training from scratch
if(training_from_base):
    #change the starting_loss if continuing on a saved model.
    starting_loss=np.inf
    # train the model
    model_transfer = train(200, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, '/home/paolo/models/model_transfer.pt', starting_loss)

    # load the model that got the best validation accuracy
    model_transfer.load_state_dict(torch.load('/home/paolo/models/model_transfer.pt'))

In [ ]:
#reload and continue training
if(update_training):   
    model_transfer.load_state_dict(torch.load('/home/paolo/models/model_transfer.pt'))
    if use_cuda:
        model_transfer.cuda()

    #change the starting_loss if continuing on a saved model.
    starting_loss=1.778868
    # train the model
    model_transfer = train(30, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, '/home/paolo/models/model_transfer.pt', starting_loss)

    # load the model that got the best validation accuracy
    model_transfer.load_state_dict(torch.load('/home/paolo/models/model_transfer.pt'))

In [ ]:
import weightwatcher as ww

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('/home/paolo/models/model_transfer.pt'))

def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.


    
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        #if use_cuda:
        #    data, target, model = data.cuda(), target.cuda(), model.cuda()
        data, target, model = data.cpu(), target.cpu(), model.cpu()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    
    watcher = ww.WeightWatcher(model)
    results = watcher.analyze(plot=True)
    summary = watcher.get_summary()
    details = watcher.get_details()
    
    print("Weight Watcher Summary")
    print(watcher.print_results)

# call test function    
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

In [ ]:
model_transfer.load_state_dict(torch.load('/home/paolo/models/model_transfer.pt'))

transform = transforms.Compose([
         transforms.Resize(256),                    
         transforms.CenterCrop(224),                
         transforms.ToTensor()
         ])

def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

classes = train_data.classes

testimage = image_loader(transform, 'oneoff/chrismark.jpg')

def test(timage, model, use_cuda): 
    model.eval()
    # move to GPU
    if use_cuda:
        model.cuda()
        timage = timage.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    testoutput = model(timage)
    # convert output probabilities to predicted class
    pred = testoutput.data.softmax(dim=1).topk(5)
    # compare predictions to true label
    indices = pred[1]
    for ind in range(len(indices[0])):
        print('{}. {:>6.2%} - {} '.format(ind, pred[0][0][ind], classes[indices[0][ind]]))
    #print('Results : {} {} {} {} {}'.format(
    #    classes[pred[1][0]]), classes[pred[1][1]], classes[pred[1][2]], classes[pred[1][3]], classes[pred[1][4]]
    #     )


# call test function    
test(testimage, model_transfer, use_cuda)
